# VOLTANDO UMA PASTA PARA TRAZ

In [ ]:
import os
os.chdir("../")

# BAIXANDO REPLAY

In [ ]:
from modules.downloader import FileDownloader

# https://www.opendota.com/matches/8319425354

FD = FileDownloader()
FD.download(
    url="http://replay153.valve.net/570/8319425354_1780917831.dem.bz2",
    download_path="./dota2_replay.dem.bz2"
)

# CONEXÃO COM BANCO

In [ ]:
from modules.database.postgres_connection import PostgresConnection
from modules.database.database_manager import DatabaseManager

host    = "localhost"
db      = "comexstat"
user    = "postgres"
pwd     = "postgres"  # Substitua pela sua senha real

# Certifique-se de definir as variáveis host, db, user, pwd antes de usar
conn = PostgresConnection(host, db, user, pwd)
conn.connect()
db_manager = DatabaseManager(conn)



# EXTRAINDO ARQUIVO DO REPLAY

In [ ]:
from modules.extractor import BZ2Extractor

bz2_extractor = BZ2Extractor(
    source_path="./dota2_replay.dem.bz2",
    dest_dir="./dota2_replay.dem"
)
bz2_extractor.extract()

# PARSEANDO DADOS DO ARQUIVO .DEM

In [ ]:
import requests

url = "http://localhost:5700/upload"  # Altere para o endereço correto do seu servidor
file_path = "dota2_replay.dem"

with open(file_path, "rb") as f:
    response = requests.post(url, data=f.read())

print("Status code:", response.status_code)
# print("Resposta:", response.text)

with open("saida.json", "w") as f:
    f.write(response.text)

# DADOS DO MEU CANAL YOUTUBE

In [ ]:
#!/usr/bin/env python3

"""
Retrieve the authenticated user's uploaded YouTube videos.
Usage:
    python my_uploads.py
"""

import os
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google_auth_oauthlib.flow import InstalledAppFlow

CLIENT_SECRETS_FILE = 'client_secret.json'
SCOPES = ['https://www.googleapis.com/auth/youtube.readonly']
API_SERVICE_NAME = 'youtube'
API_VERSION = 'v3'

def get_authenticated_service():
    """Authenticate and return a YouTube API service instance."""
    print("1. Authenticating with YouTube API...")
    flow = InstalledAppFlow.from_client_secrets_file(CLIENT_SECRETS_FILE, SCOPES)
    print("2. Please visit the following URL to authorize the application:")
    credentials = flow.run_local_server(port=8090)
    print("3. Authentication successful!")
    return build(API_SERVICE_NAME, API_VERSION, credentials=credentials)

def get_my_uploads_playlist_id(youtube):
    """
    Retrieve the uploads playlist ID for the authenticated user's channel.
    Returns:
        str: The uploads playlist ID, or None if not found.
    """
    channels_response = youtube.channels().list(
        mine=True,
        part='contentDetails'
    ).execute()

    for channel in channels_response.get('items', []):
        return channel['contentDetails']['relatedPlaylists']['uploads']
    return None

def list_my_uploaded_videos(youtube, uploads_playlist_id, max_results=5):
    """
    Print the titles and IDs of videos in the uploads playlist.
    Args:
        youtube: Authenticated YouTube API service instance.
        uploads_playlist_id (str): The uploads playlist ID.
        max_results (int): Number of results per page.
    """
    request = youtube.playlistItems().list(
        playlistId=uploads_playlist_id,
        part='snippet',
        maxResults=max_results
    )

    print(f'Videos in playlist {uploads_playlist_id}:')
    while request:
        response = request.execute()
        for item in response.get('items', []):
            title = item['snippet']['title']
            video_id = item['snippet']['resourceId']['videoId']
            print(f'{title} ({video_id})')
        request = youtube.playlistItems().list_next(request, response)

if __name__ == '__main__':
    try:
        youtube = get_authenticated_service()
        uploads_playlist_id = get_my_uploads_playlist_id(youtube)
        if uploads_playlist_id:
            list_my_uploaded_videos(youtube, uploads_playlist_id)
        else:
            print('No uploaded videos playlist found for this user.')
    except HttpError as e:
        print(f'An HTTP error {e.resp.status} occurred:\n{e.content}')

In [ ]:
request = youtube.playlistItems().list(
    playlistId=uploads_playlist_id,
    part='snippet',
    maxResults=5
)

print(f'Videos in playlist {uploads_playlist_id}:')
response = request.execute()
for item in response.get('items', []):
    title = item['snippet']['title']
    video_id = item['snippet']['resourceId']['videoId']
    print(f'{title} ({video_id})')
request = youtube.playlistItems().list_next(request, response)

# SCRAPPING


### Precisa do ID do video (Super Rapido)

In [ ]:
# https://pypi.org/project/yt-api-wrapper/

from yt_api_wrapper import YouTubeAPIWrapper # Import the library
yt = YouTubeAPIWrapper() # Create API wrapper

video = yt.get_video_info('dQw4w9WgXcQ') # Get video info with one line

results = yt.search_videos('python course') # Search videos

In [ ]:
"""
video_id
title
description
author
channel_id
length_seconds
view_count
keywords
upload_date
category
thumbnail_url
"""

In [ ]:
# https://pypi.org/project/ytwrap/

import ytwrap

video = ytwrap.YTVideoClient()
comment = ytwrap.YTCommentClient()

# Get latest video
latest = video.get_latest_video('CHANNEL_ID')

# Analyze comments
stats = comment.count_comments_and_replies('VIDEO_ID')

### MELHOR ATÉ AGORA

Get Video Information

In [ ]:
from ytbpy import video

# Get details about a video using URL or ID
video_info = video.get_video_info('https://www.youtube.com/watch?v=rZ6luwT8kuc')
# OR
video_info = video.get_video_info('rZ6luwT8kuc')

print(video_info['title'])
print(video_info['duration'])
print(video_info['views_count'])

Using OEmbed API

In [ ]:
from ytbpy import video

# Get details about a video using URL or ID
video_info = video.get_video_info_oembed('https://www.youtube.com/watch?v=rZ6luwT8kuc')
# OR
video_info = video.get_video_info_oembed('rZ6luwT8kuc')

print(video_info['title'])

Search YouTube

In [ ]:
from ytbpy import search

# Search YouTube videos
results = search.search_youtube('python tutorial', max_results=5)

for video in results:
    print(f"{video['title']} - {video['url']}")

Get Playlist Information

In [ ]:
from ytbpy import playlist

# Get all videos in a playlist
playlist_info = playlist.get_playlist_info('https://www.youtube.com/playlist?list=PLOU2XLYxmsIJQPs-2nnII-n59vYoxKsmq')

print(f"Playlist: {playlist_info['title']}")
print(f"Videos: {playlist_info['video_count']}")

for video in playlist_info['videos']:
    print(video['title'])

Get Channel Information

In [3]:
from ytbpy import channel, video

# Get channel info and recent videos
channel_info = channel.get_channel_info('https://www.youtube.com/@DotaSupport')

print(f"Channel: {channel_info['title']}")

# List recent videos
for video in channel_info['videos']:
    print(video['title'])

Channel: Dota Persona
Insane Ganking Boxi MARCI Soft Support Gameplay Patch 7.39b - Dota 2 Support
Planet EARTHSHAKER Soft Support Gameplay Patch 7.39b - Dota 2 Support
Kataomi CRYSTAL MAIDEN Hard Support Gameplay Patch 7.39b - Dota 2 Support
Y' ABADDON The Healer Hard Support Gameplay Patch 7.39b - Dota 2 Support
Omar DARK WILLOW Soft Support Gameplay Patch 7.39b - Dota 2 Support
PUCK Support ??? Tofu NEW META In The Tournament - Dota 2 Support
Whitemon NAGA SIREN Hard Support Gameplay Patch 7.39b - Dota 2 Support
Y' UNDYING Hard Support Gameplay Patch 7.39b - Dota 2 Support
Tobi VENOMANCER The Poisonous Hard Support Gameplay Patch 7.39b - Dota 2 Support
Malady WARLOCK Hard Support Gameplay Patch 7.39b - Dota 2 Support


In [ ]:
video_info = video.get_video_info('https://www.youtube.com/watch?v=4rGsQQYRmtY')

In [ ]:
video_info

# SUMARIZAÇÃO DE VIDEOS DO YOUTUBE

In [ ]:
# https://pypi.org/project/llm-fragments-youtube/